<a href="https://colab.research.google.com/github/mansh2060/QA-bot/blob/main/QAbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import wget
from transformers import DPRContextEncoder,DPRContextEncoderTokenizer,DPRQuestionEncoder,DPRQuestionEncoderTokenizer
import torch
import numpy as np
from transformers import AutoTokenizer,AutoModelForCausalLM
from pinecone import Pinecone,ServerlessSpec,PodSpec

In [ ]:
filename='companypolicies.txt'
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'
wget.download(url,out=filename)

'companypolicies (1).txt'

In [ ]:
#file-read + data - list + data -space eliminate  = return paragraph
def read_file(filename):
    with open(filename,'r',encoding='utf-8') as files:
        file=files.read()

    paragraphs=file.split('\n')
    paragraphs=[para.strip() for para in paragraphs if len(para.strip()) > 0 ]
    return paragraphs
paragraphs=read_file('companypolicies.txt')

In [ ]:
model_name="facebook/dpr-ctx_encoder-single-nq-base"

Used DPR for tokenization,embeddings,question answer answering instead of OPENAI API

In [ ]:
def encode_contexts(text_list):
    embeddings=[]
    context_tokenizer=DPRContextEncoderTokenizer.from_pretrained(model_name)
    context_encoder=DPRContextEncoder.from_pretrained(model_name)
    for text in text_list:
        tokens=context_tokenizer(text,padding=True,return_tensors="pt",truncation=True,max_length=256)
        vectors=context_encoder(**tokens)
        embeddings.append(vectors.pooler_output)
    return torch.cat(embeddings).detach().numpy()
context_embedding=encode_contexts(paragraphs)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
You are using a model of type bert to instantiate a model of type dpr. This is not supported for all configurations of models and can yield errors.
Some weights of DPRContextEncoder were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert_model.embeddings.LayerNorm.bias', 'bert_model.embeddings.LayerNorm.weight', 'bert_model.embeddings.position_embeddings.weight', 'bert_model.embeddings.token_type_embeddings.weight', 'bert_model.embeddings.word_embeddings.weight', 'bert_model.encoder.layer.0.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.0.attention.output.LayerNorm.weight', 'bert_model.encoder.layer.0.attention.output.dense.bias', 'b

In [ ]:
len(context_embedding)

76

In [ ]:
context_embedding.shape

(76, 768)

In [ ]:
context_embedding

array([[ 1.2332821 , -1.1841292 ,  0.03847372, ...,  0.24722798,
         1.3102813 , -1.9066025 ],
       [ 0.97840405, -1.019581  ,  0.24952444, ...,  0.41552603,
         1.2107834 , -1.6416715 ],
       [ 0.9553924 , -1.1559378 ,  0.2559863 , ...,  0.48858047,
         1.2516516 , -1.7455238 ],
       ...,
       [ 0.7808161 , -1.0014777 ,  0.32689705, ...,  0.27574834,
         1.1540724 , -1.6400774 ],
       [ 0.88111687, -0.99578106,  0.20064867, ...,  0.51199406,
         1.2861305 , -1.7275058 ],
       [ 0.84801227, -0.9993715 ,  0.35512507, ...,  0.31374866,
         1.4298874 , -1.6731874 ]], dtype=float32)

Used PineCone for vector database

In [ ]:
import os
use_serverless=True

In [ ]:
api_key=os.environ.get('PINECONE_API_KEY')
pc=Pinecone(api_key="pcsk_3LLvfq_RmMBF385Va33QedCqyG6cU9HuvANvMbXnLXYzzP25wK4Lmyx8jRadnkUQ4tuHWN")

In [ ]:
if use_serverless:
  spec=ServerlessSpec(cloud='aws',region='us-east-1')
else:
  spec=PodSpec(environment='environment')

In [ ]:
index_name="ragdatabase"
if index_name in pc.list_indexes().names():
   pc.delete_index(index_name)

In [ ]:
index_name = "ragdatabase"
if index_name not in pc.list_indexes():
    pc.create_index(index_name,
                    dimension=768,
                    metric="cosine",
                    spec=ServerlessSpec(
                    cloud='aws',
                    region='us-east-1'
                     )
                    )
index = pc.Index(index_name)
ids = [f"id-{i}" for i in range(len(context_embedding))]

data = [(ids[i], context_embedding[i].tolist()) for i in range(len(context_embedding))]
index.upsert(vectors=data)
print("Vectors uploaded successfully to Pinecone!")


Vectors uploaded successfully to Pinecone!


In [ ]:
question_encoder=DPRQuestionEncoder.from_pretrained(model_name)
question_tokenizer=DPRQuestionEncoderTokenizer.from_pretrained(model_name)

You are using a model of type bert to instantiate a model of type dpr. This is not supported for all configurations of models and can yield errors.
Some weights of DPRQuestionEncoder were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert_model.embeddings.LayerNorm.bias', 'bert_model.embeddings.LayerNorm.weight', 'bert_model.embeddings.position_embeddings.weight', 'bert_model.embeddings.token_type_embeddings.weight', 'bert_model.embeddings.word_embeddings.weight', 'bert_model.encoder.layer.0.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.0.attention.output.LayerNorm.weight', 'bert_model.encoder.layer.0.attention.output.dense.bias', 'bert_model.encoder.layer.0.attention.output.dense.weight', 'bert_model.encoder.layer.0.attention.self.key.bias', 'bert_model.encoder.layer.0.attention.self.key.weight', 'bert_model.encoder.layer.0.attention.self.query.bias', 'bert_model.encoder.layer.0.attention.self.query.weight', 'bert_model.encod

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [ ]:
def generate_answer(question):
    inputs = tokenizer(question, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0,
                                 num_beams=4, early_stopping=True,pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return answer

# Testing ***QA*** Bot from generate answer function

In [ ]:
question = "what is Smoking policy?"
answer = generate_answer(question)
print(answer)

what is Smoking policy?

Smoking is defined as "the use of tobacco products, including cigarettes, cigars, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes, pipes,


In [ ]:
question = "what is Recruitment policy?"
answer = generate_answer(question)
print(answer)

what is Recruitment policy?

Recruitment policy is the process by which a recruiter is able to identify potential candidates for a position. Recruitment policy is the process by which a recruiter is able to identify potential candidates for a position. Recruitment policy is the process by which a recruiter is able to identify potential candidates for a position. Recruitment policy is the process by which a recruiter is able to identify potential candidates for a position. Recruitment policy is the process by which a recruiter is able to identify potential candidates for a position. Recruitment policy is the process by which a recruiter is able to identify potential candidates for a position. Recruitment policy is the process by which


In [ ]:
question = "what is Internet and Email policy?"
answer = generate_answer(question)
print(answer)

what is Internet and Email policy?

Internet and email policy is the responsibility of the user. It is the responsibility of the user to ensure that the information provided on the Internet is accurate and up-to-date. It is also the responsibility of the user to ensure that the information provided on the Internet is accurate and up-to-date.

How do I register for an account?

You can register for an account at any time by clicking on the "Register Now" link on the right side of the page.

How do I sign up for an account?

You can sign up for an account at any time by clicking on the "Sign Up Now" link on the right side of the page.


In [ ]:
question = "what is Drug and Alcohol policy?"
answer = generate_answer(question)
print(answer)

what is Drug and Alcohol policy?

Drug and alcohol policy is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is the policy of the government. It is


In [ ]:
question = "what is Health and Safety Policy?"
answer = generate_answer(question)
print(answer)

what is Health and Safety Policy?

Health and Safety Policy (HSP) is a federal law that governs the health and safety of the U.S. population.

HSP is a federal law that governs the health and safety of the U.S. population. The U.S. Department of Health and Human Services (HHS) is responsible for administering HSP.

HSP is a federal law that governs the health and safety of the U.S. population. The U.S. Department of Health and Human Services (HHS) is responsible for administering HSP. The U.S. Department of Agriculture (USDA) is responsible for administering HSP.

HSP is a federal law
